# Finding Causality in Big Data

## Import required libraries and data

In [3]:
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import binom, hypergeom, gaussian_kde
from scipy.stats import ttest_ind
import math
from numpy import linalg
import scipy.spatial.distance as ssdist
import statsmodels.genmod.generalized_linear_model as sm

# Attributes:

* school - student's school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)
* sex - student's sex (binary: "F" - female or "M" - male)
* age - student's age (numeric: from 15 to 22)
* address - student's home address type (binary: "U" - urban or "R" - rural)
* famsize - family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
* Pstatus - parent's cohabitation status (binary: "T" - living together or "A" - apart)
* Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
* Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
* Mjob - mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
* Fjob - father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
* reason - reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
* guardian - student's guardian (nominal: "mother", "father" or "other")
* traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
* studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
* failures - number of past class failures (numeric: n if 1<=n<3, else 4)
* schoolsup - extra educational support (binary: yes or no)
* famsup - family educational support (binary: yes or no)
* paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
* activities - extra-curricular activities (binary: yes or no)
* nursery - attended nursery school (binary: yes or no)
* higher - wants to take higher education (binary: yes or no)
* internet - Internet access at home (binary: yes or no)
* romantic - with a romantic relationship (binary: yes or no)
* famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
* freetime - free time after school (numeric: from 1 - very low to 5 - very high)
* goout - going out with friends (numeric: from 1 - very low to 5 - very high)
* Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
* Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
* health - current health status (numeric: from 1 - very bad to 5 - very good)
* absences - number of school absences (numeric: from 0 to 93)

### These grades are related with the course subject, Math or Portuguese:
* G1 - first period grade (numeric: from 0 to 20)
* G2 - second period grade (numeric: from 0 to 20)
* G3 - final grade (numeric: from 0 to 20, output target)

In [5]:
# 1. read data
df = pd.read_csv("/dbfs/FileStore/tables/993e24dq1477058574109/student_mat-be366.csv",sep= ";").drop(['G1','G2'],1)
print(df.columns)
print(df.shape) # 382 students
df.head()

In [6]:
# 2. convert all attributes into integer
# to do: store mappings for each attribute in a dictionary
for col in df:
    if (df[col].dtypes == object) == True:
        new_map = dict(zip(df[col].unique(),range(len(df[col].unique()))))
        df[col] = df[col].map(new_map)
df.head()

* Outcome: Final grade
* Treatment: paid - extra paid classes within the course subject (binary: yes or no)
* Covariates: all the other attributes

## Define treatment, outcome and covariates

In [9]:
# treatment
TREAT = 'paid'
Tr = df[TREAT]

# outcome
OUT = 'G3'

# 1 Identify covariates from observational data
## 1.1 Difference-in-means: outcome variable

* Effectiveness of treatment (effect on RE78 expression)
* Question: does treatment affect RE78 expression (outcome)?

In [11]:
df.groupby(TREAT).size()

In [12]:
df.groupby(TREAT)[OUT].mean().reset_index()

In [13]:
# t-statistic & p-value for difference in outcome of two groups
def hypothesisTest(data, attribute, group):
    x = data[attribute][df[group]== 1]
    y = data[attribute][df[group]== 0]
    # t-statistic & p-value for difference in outcome of two groups
    t = ttest_ind(x, y)[0]
    p = ttest_ind(x, y)[1]
    # Confidence intervals
    cm = sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y))
    CI = str(round(cm.tconfint_diff(usevar='unequal')[0]))+' - '+str(round(cm.tconfint_diff(usevar='unequal')[1]))
    # Cohen's d
    pooledvar = math.sqrt((pow(x.std(),2) + (pow(y.std(),2)))/2)
    d = (x.mean()-y.mean()) / pooledvar
    # create dataframe
    tablelist = []
    tablerow = [attribute,x.mean()-y.mean(),t,p,CI,d]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','Mean Difference','t-value','p-value','95% Confidence Intervals',"Cohen's d"]

    return out

In [14]:
# covariates
b = pd.DataFrame()
for att in df.drop([TREAT,OUT],1):
    a = []
    a = hypothesisTest(df, att, TREAT)
    b = pd.concat([b,a],0)
b.sort('p-value')

In [15]:
# covariates
COV = np.array(b.Attribute[b['p-value'] <= 0.05])
X = df[COV]
X.head()

## 2.1 Propensity score matching

In [17]:
####### Using GLM
glm_binom = sm.GLM(Tr, X, family=sm.families.Binomial())
res = glm_binom.fit()
print res.summary()
propensityScoreGLM = res.fittedvalues
df_new = pd.concat([df[COV],df[OUT],propensityScoreGLM],1)
df_new.columns = np.append(COV, [OUT,'Propensity Score'], axis=None )
print '\n'+ 'Propensity Scores added: '
df_new.head()

## Matching 

* optimal matching 
* greedy nearest neighbor matching without replacement
* greedy nearest neighbor matching without replacement within specified caliper widths
* nearest neighbor matching with replacement
* nearest neighbor matching with replacement within specified caliper widths

In [19]:
def Match(groups, propensity, caliper , replace = False):
        
    # Code groups as 0 and 1
    groups = groups == groups.unique()[0]
    N = len(groups)
    N1 = groups[groups == 1].index; N2 = groups[groups == 0].index
    g1, g2 = propensity[groups == 1], propensity[groups == 0]
    # Check if treatment groups got flipped - the smaller should correspond to N1/g1
    if len(N1) > len(N2):
       N1, N2, g1, g2 = N2, N1, g2, g1
        
    # Randomly permute the smaller group to get order for matching
    morder = np.random.permutation(N1)
    matches = {}
    
    for m in morder:
        dist = abs(g1[m] - g2)
        if (dist.min() <= caliper) or not caliper:
            matches[m] = dist.argmin()    # Potential problem: check for ties
            if not replace:
                g2 = g2.drop(matches[m])
    return (matches)


def whichMatched(matches, data):
  
    tr = matches.keys()
    ctrl = matches.values()
    temp = pd.concat([data.ix[tr], data.ix[ctrl]])

    return temp

In [20]:
# one-to-one without caliper, without replacement
matches = Match(Tr, propensityScoreGLM, caliper = 0, replace = False)
temp = whichMatched(matches, df)

PS_matched = temp.groupby(temp.index).first()
PS_matched.groupby(TREAT).size().reset_index()

In [21]:
# Exercise: Run one-to-one matching with fixed 0.05 caliper, without replacement
matches = Match(Tr, propensityScoreGLM, caliper = 0.05, replace = False)
temp = whichMatched(matches, df)

PS_matched1 = temp.groupby(temp.index).first()
PS_matched1.groupby(TREAT).size().reset_index()

In [22]:
# Exercise: Run one-to-one matching with logit caliper, without replacement
matches = Match(Tr, propensityScoreGLM, caliper = 'logit', replace = False)
temp = whichMatched(matches, df)

PS_matched2 = temp.groupby(temp.index).first()
PS_matched2.groupby(TREAT).size().reset_index()

In [23]:
# Exercise: Run one-to-one matching with logit caliper, with replacement
matches = Match(Tr, propensityScoreGLM, caliper = 'logit', replace = True)
temp = whichMatched(matches, df)

PS_matched3 = temp.groupby(temp.index).first()
PS_matched3.groupby(TREAT).size().reset_index()

# 4 Evaluating matching methods

In [25]:
f = plt.figure(figsize=(15,5))

PS_matched_norm = (PS_matched[COV] - PS_matched[COV].mean()) / (PS_matched[COV].max() - PS_matched[COV].min())
PS_matched_mean = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched[TREAT]==1].mean() - PS_matched_norm[PS_matched[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched1[COV] - PS_matched1[COV].mean()) / (PS_matched1[COV].max() - PS_matched1[COV].min())
PS_matched_mean1 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched1[TREAT]==1].mean() - PS_matched_norm[PS_matched1[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched2[COV] - PS_matched2[COV].mean()) / (PS_matched2[COV].max() - PS_matched2[COV].min())
PS_matched_mean2 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched2[TREAT]==1].mean() - PS_matched_norm[PS_matched2[TREAT]==0].mean())}).mean()[0]

PS_matched_norm = (PS_matched3[COV] - PS_matched3[COV].mean()) / (PS_matched3[COV].max() - PS_matched3[COV].min())
PS_matched_mean3 = pd.DataFrame({'mean':abs(PS_matched_norm[PS_matched3[TREAT]==1].mean() - PS_matched_norm[PS_matched3[TREAT]==0].mean())}).mean()[0]

x = [PS_matched.shape[0],PS_matched1.shape[0],PS_matched2.shape[0],PS_matched3.shape[0]]
y = [PS_matched_mean ,PS_matched_mean1,PS_matched_mean2,PS_matched_mean3]
colors = ['r','b','g','c','k','m','y']
MatchingMethod = ['1-1 woutC woutR','1-1 w0.05C woutR','1-1 wLogitC woutR','1-1 wLogitC wR']

j = 0
for i in x:
    plt.scatter(i, y[j], s=120, c=colors[j], label=MatchingMethod[j])
    j += 1
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),ncol=1, fancybox=True, fontsize=8)
    
plt.xlabel('Sample Size')
plt.ylabel('Balance Metric')

display(f)

* The comparison of sample size and balance between different matchings show that the Green dot that represents 1-1 matching with Logit caliper and without replacement gives the highest sample size while its balance competes with the blue dot by being only 0.03 higher. 
* Therefore, we use PS_matched2 as the prefered matched set.

## 5 Estimate treatment effects

Estimate average treatment effect  on the treated

### 5.1 Computes ATT using difference in means

In [28]:
# t-statistic & p-value for difference in outcome of two groups
def hypothesisTestOutcome(data, attribute, group):
    x = data[attribute][df[group]== 1]
    y = data[attribute][df[group]== 0]
    # t-statistic & p-value for difference in outcome of two groups
    t = ttest_ind(x, y)[0]
    p = ttest_ind(x, y)[1]
    # Confidence intervals
    cm = sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y))
    CI = str(round(cm.tconfint_diff(usevar='unequal')[0]))+' - '+str(round(cm.tconfint_diff(usevar='unequal')[1]))
    # Cohen's d
    pooledvar = math.sqrt((pow(x.std(),2) + (pow(y.std(),2)))/2)
    d = (x.mean()-y.mean()) / pooledvar
    # create dataframe
    tablelist = []
    tablerow = [attribute,x.mean()-y.mean(),((x.mean()-y.mean())/df[attribute].max()),t,p,d]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','Outcome Mean Difference','Outcome Mean Difference %','t-value','p-value',"Cohen's d"]

    return out

In [29]:
# Matched outcome
Y0_PS_matched = PS_matched2.G3[df.paid == 0]
Y1_PS_matched = PS_matched2.G3[df.paid == 1]

print 'Differene in means: '+ str(int(Y1_PS_matched.mean() - Y0_PS_matched.mean())) 

* We observe that the initial difference in score for students who took the paid classes and didn't is equal to zero after matching, while it was 1 point score (out of 20) before matching.
* Therefore, we can conclude that the extra paid classes did not have an effect in impro